In [1]:
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
import Generate_word2vec_features
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
import pandas as pd
import gensim
import os
import sys
import RunModels
sys.path.append( os.path.join(".."))
from UtilityFunctions import CommonHelpers,PreprocessHelpers,FeatureEngineering
import numpy as np
import pandas as pd
import re

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/nitishdewan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/nitishdewan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/nitishdewan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/nitishdewan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
# Constants
book_pickle_path="../data/preprocessed/books_english.pickle"
name_pickle_path = "../data/preprocessed/names_english.pickle"

In [3]:
def get_number_id(file_name):
    regex = r"([a-z_A-Z]+)(\d+)"
    match = re.search(regex, file_name)
    if match!=None:
        return match.group(2)

In [4]:
list_of_words = CommonHelpers.load_pickle(book_pickle_path)
book_names =  CommonHelpers.load_pickle(name_pickle_path)

In [ ]:
list_of_words = CommonHelpers.load_pickle(book_pickle_path)

In [5]:
df = pd.read_csv("../data/pg_catalog.csv")
df["category"] = df["LoCC"].str[:1]
category_list_train = [int(get_number_id(each)) for each in book_names]
selected_books = df[df["Text#"].isin(category_list_train)]
categories=[]
titles=[]
for each in category_list_train:
    categories.append(selected_books[selected_books["Text#"]==each]["category"].values[0])
    titles.append(selected_books[selected_books["Text#"]==each]["Title"].values[0])

/Users/nitishdewan/downloads/ls/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
# For testing
'''
number = 420
print(list_of_words[number][:100])
print(">>>>>>>>>>>>>>>.")
print(book_names[number],categories[number],titles[number],category_list_train[number])
'''

In [ ]:
# Google Word2Vec book vectors
gen = Generate_word2vec_features.Word2Vec_Features()
gen.load_data(cache=True)
train_vec =np.nan_to_num( gen.book_vectors)



In [ ]:
# BoW
# load the final vectors of size 1400*X (vary X ) into variable train_vec

In [ ]:
# TF-IDF
# load the final vectors of size 1400*X (vary X ) into variable train_vec
VECTOR_PICKLE_LOC = "../data/vectors"
combined_vector_1 = np.concatenate((CommonHelpers.load_pickle(os.path.join(VECTOR_PICKLE_LOC,"TfIDF_1_gram_1.pickle")), CommonHelpers.load_pickle(os.path.join(VECTOR_PICKLE_LOC,"TfIDF_1_gram_2.pickle"))), axis = 0)
tfidf_1gram_combined = np.concatenate((combined_vector_1, CommonHelpers.load_pickle(os.path.join(VECTOR_PICKLE_LOC,"TfIDF_1_gram_3.pickle"))), axis = 0)
tfidf_1gram_combined.shape

In [ ]:
train_vec = tfidf_1gram_combined

In [6]:
# Custom Word2Vec
# load the final vectors of size 1400*X (vary X ) into variable train_vec

gen = Generate_word2vec_features.Word2Vec_Features()
gen.init_data()
gen.load_data(cache=False)
train_vec =np.nan_to_num(gen.book_vectors)


Init data Started
Init data completed
Loading data
1400 <class 'list'>


/var/folders/v1/xkw5ylbj38z8m09c8n_mzf880000gn/T/ipykernel_2936/2076003803.py:6: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  gen.load_data(cache=False)


Loading data complete
Saving to pickles done


In [ ]:
split_obj = RunModels.Split_Data(train_vec,categories)
split_obj.get_split()

In [7]:
names = [
    "Nearest Neighbors",
    "RBF SVM",
    "Random Forest",
    "Neural Net",
    "Naive Bayes",
    "Logistic Regression",
    "Logistic Regression One-vs-Rest",
    "LinearSVC One-vs-Rest"
]
classifiers = [
    KNeighborsClassifier(3),
    SVC(gamma=2, C=1),
    RandomForestClassifier(n_estimators=400),
    MLPClassifier(alpha=0.05, max_iter=10000),
    GaussianNB(),
    LogisticRegression(multi_class='multinomial', random_state=0, max_iter=500),
    LogisticRegression(multi_class='ovr', random_state=42, max_iter=500),
    OneVsRestClassifier(SVC()),
]

In [8]:
for name, clf in zip(names, classifiers):
    model = RunModels.Run_Model(clf,split_obj)
    model.run_model()
    result = model.get_predictions()
    matrix,accuracy = model.get_metrics()
    print(name,accuracy)

Nearest Neighbors 0.4642857142857143
RBF SVM 0.16785714285714284
Random Forest 0.5964285714285714
Neural Net 0.5964285714285714
Naive Bayes 0.225
Logistic Regression 0.5571428571428572
Logistic Regression One-vs-Rest 0.5821428571428572
LinearSVC One-vs-Rest 0.5928571428571429
